In [1]:
#first we need to handle all the important libraries we will need in this project
#note that different versions of python instalation will not support most of the libraries such as tensorflow.contrib
#so it is for this reason we need to install only the python version which is compatible with this library
#loading all the modules neccesary to run our algorithms
#make sure to install all the below libraries for better compatibility and perfomance
#keras 2.1.4
#tensorflow 1.14.0
#tensorflow_probablity 0.7.0
#scipy
#matplotlib
#python 3.6 or 3.7

In [2]:
from models.BoundedFullyNonLinear import *
from models.Merton import *
from models.NoLeverage import *
from models.OneAsset import *
from models.MongeAmpere import *
from models.SemiBounded import *
from models.SemiUnbounded import *
from models.Burgers import *


C:\Users\sojore\anaconda3\envs\py37s\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sojore\anaconda3\envs\py37s\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sojore\anaconda3\envs\py37s\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sojore\anaconda3\envs\py37s\lib\site-packages\tensorflow\pyth

In [3]:
from networks.FeedForwardU import *
from networks.FeedForwardUDU import *
from networks.FeedForwardUZ import *
from networks.FeedForwardUZDZ import *
from networks.FeedForwardGam import *


In [4]:
from solvers.FullNonLinearMEDBDP  import *
from solvers.FullNonLinearM2DBDPGPU  import *
from solvers.FullNonLinearMDBDPGPU import *
from solvers.FullNonLinearBaseGPU import *
from solvers.FullNonLinearPWG import *
from solvers.SemiLinearBaseGPU import *
from solvers.Splitting import *
from solvers.HPW import *
from solvers.SemiHJE import *
import matplotlib
from matplotlib import pyplot as plt

In [5]:
#pip install matplotlib

In [5]:
import numpy as np
import tensorflow as tf
import os
import solvers  as solv
import networks as net
import models as mod
import multiprocessing
import sys
import time

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

_MULTIPROCESSING_CORE_COUNT = multiprocessing.cpu_count()
print("args", sys.argv)

d = 1
xInit= np.ones(d,dtype=np.float32) 
nbLayer= 2  
rescal=1.
T=1.

batchSize= 1000
batchSizeVal= 10000
num_epoch=400 
num_epochExtNoLast =10
num_epochExtLast= 200
initialLearningRateLast = 1e-2
initialLearningRateNoLast = 1e-3
nbOuterLearning =10
nTest = 1

lamb = np.array([1.5], dtype=np.float32)
eta = 0.5
theta = np.array([0.4], dtype=np.float32) 
gamma = np.array([0.2], dtype=np.float32) 
kappa = np.array([1.], dtype=np.float32)
sigma = np.array([1.], dtype=np.float32)
nbNeuron = d + 10
sigScal =   np.array([1.], dtype=np.float32)

muScal = np.array([np.sum(theta*lamb)])
layerSize= nbNeuron*np.ones((nbLayer,), dtype=np.int32) 

# create the model
model = mod.ModelMerton(xInit, muScal, sigScal, T, theta, lamb, eta)

print(" WILL USE " + str(_MULTIPROCESSING_CORE_COUNT) + " THREADS ")
print("REAL IS ", model.Sol(0.,xInit), " DERIV", model.derSol(0.,xInit))

theNetwork = net.FeedForwardGam(d,layerSize,tf.nn.tanh)
theNetworkGam = net.FeedForwardGam(d,layerSize,tf.nn.tanh)

ndt = [ (120,3) ] 

print("PDE Merton MDBDP  Dim ", d, " layerSize " , layerSize,  " rescal " ,rescal, "T ", T , "batchsize ",batchSize, " batchSizeVal ", batchSizeVal, "num_epoch " , num_epoch, " num_epochExtNoLast ", num_epochExtNoLast  , "num_epochExtLast " , num_epochExtLast, "VOL " , sigScal, "initialLearningRateLast" , initialLearningRateLast , "initialLearningRateNoLast " , initialLearningRateNoLast)

# nest on ndt
for indt  in ndt:

    print("NBSTEP",indt)
    # create graph
    resol =  solv.PDEFNLSolve2OptZGPU(model, T, indt[0], indt[1], theNetwork , theNetworkGam, initialLearningRateLast=initialLearningRateLast, initialLearningRateNoLast = initialLearningRateNoLast)
        
    baseFile = "MertonMDBDPd"+str(d)+"nbNeur"+str(layerSize[0])+"nbHL"+str(len(layerSize))+"ndt"+str(indt[0])+str(indt[1])+"eta"+str(int(eta*100))
    plotFile = "C:\\Users\\sojore\\Pictures/"+baseFile
    saveFolder = "save/"

    Y0List = []
    for i in range(nTest):
        # train
        t0 = time.time()
        Y0, Z0, Gamma0    = resol.BuildAndtrainML( batchSize, batchSizeVal, num_epochExtNoLast=num_epochExtNoLast, num_epochExtLast= num_epochExtLast ,num_epoch=num_epoch,  nbOuterLearning=nbOuterLearning, thePlot= plotFile , baseFile = baseFile, saveDir= saveFolder)
        t1 = time.time()
        print(" NBSTEP", indt, " EstimMC Val is " , Y0,  " REAL IS ", model.Sol(0.,xInit),    " Z0 ", Z0," DERREAL IS  ",model.derSol(0.,xInit), "Gamma0 " , Gamma0, t1-t0)

        Y0List.append(Y0)
        print(Y0List)

    print("Y0", Y0List)
    yList = np.array(Y0List)
    yMean = np.mean(yList)
    print(" DNT ", indt , "MeanVal ", yMean, " Etyp ", np.sqrt(np.mean(np.power(yList-yMean,2.))))


args ['C:\\Users\\sojore\\anaconda3\\envs\\py37s\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\sojore\\AppData\\Roaming\\jupyter\\runtime\\kernel-f0a75d40-f5d1-4c6b-9047-39ffcb7b3402.json']
 WILL USE 2 THREADS 
REAL IS  [-0.506617]  DERIV [[0.2533085]]
PDE Merton MDBDP  Dim  1  layerSize  [11 11]  rescal  1.0 T  1.0 batchsize  1000  batchSizeVal  10000 num_epoch  400  num_epochExtNoLast  10 num_epochExtLast  200 VOL  [1.] initialLearningRateLast 0.01 initialLearningRateNoLast  0.001
NBSTEP (120, 3)
nbStepGamStab 40
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
CREATED GAM  3
 Gam 6.559795e-05  iout 0
 Gam 1.6685612e-05  iout 1
 Gam 1.4152397e-05  iout 2
 Gam 1.2416126e-05  iout 3
 Gam 9.17798e-06  iout 7
 Gam 3.7178206e-06  iout 8
 Gam 2.6718646e-06  iout 12
 Gam 1.160165e-06  iout 16
Projection, and derivative,  Learning decrease to  0.005
 Gam 9.1444684e-07  iout 22
 Gam 9.1034974e-07  iout 39
Projection, and derivative

TypeError: Tensor objects are only iterable when eager execution is enabled. To iterate over this tensor use tf.map_fn.

In [31]:
pwd

'C:\\Users\\sojore\\Documents\\pyhthon s'

1.Mertonxxx 

-in this groups of algorithms ,we gonna train one model e.g MertonMDBDP,we then save its weights
-train another model e.g MertonM2DBDP ,freeze its hidden layers so it does not train again,
-transfer this model to load the saved weights from MertonMDBDP and compare its perfomance

In [ ]:
#lets save the weights,and the above trained model 
#saving the weights/parameters
model.save_weights('MertonMDBDP_model_types_weights.h5')#this will save the weights of the MertonMDBDP model trained above
#which we want
#to load them using a different algorithm such as MertonM2DBDP and see how it perfoms of these weights

In [ ]:
#we now going to train another model MertonM2DBDP ,save the model,load it again,freeze the hidden layers of the model,
#add an outer dense layer (output neurons=2) ,use this new model and load the MertonMDBDP_model_types_weights, to see its
#perfomance


In [6]:
# we are first executing the code below so as to get a MertonM2DBDP_model,which we can then save it,freeze the hidden layers
#then transfer the new model to load the MertonMDBDP_model_types_weights
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import solvers  as solv
import networks as net
import models as mod
import multiprocessing
import sys
import time

_MULTIPROCESSING_CORE_COUNT = multiprocessing.cpu_count()
print("args", sys.argv)

d = 1 
xInit= np.ones(d,dtype=np.float32) 
nbLayer= 2  
print("nbLayer " ,nbLayer)
rescal=1.
T=1.

batchSize= 1000
batchSizeVal= 10000
num_epoch=400 
num_epochExtNoLast =10
num_epochExtLast= 200
initialLearningRateLast = 1e-2
initialLearningRateNoLast = 1e-3
nbOuterLearning =10
nTest = 1

lamb = np.array([1.5], dtype=np.float32)
eta = 0.5
theta = np.array([0.4], dtype=np.float32) 
gamma = np.array([0.2], dtype=np.float32) 
kappa = np.array([1.], dtype=np.float32)
sigma = np.array([1.], dtype=np.float32) 
nbNeuron = d + 10
sigScal =   np.array([1.], dtype=np.float32)

muScal = np.array([np.sum(theta*lamb)])
layerSize= nbNeuron*np.ones((nbLayer,), dtype=np.int32) 
# create the model
model = mod.ModelMerton(xInit, muScal, sigScal, T, theta, lamb, eta)

print(" WILL USE " + str(_MULTIPROCESSING_CORE_COUNT) + " THREADS ")
print("REAL IS ", model.Sol(0.,xInit), " DERIV", model.derSol(0.,xInit))

theNetwork = net.FeedForwardU(d,layerSize,tf.nn.tanh)
theNetworkGam = net.FeedForwardU(d,layerSize,tf.nn.tanh)

ndt = [ (120,30) ] 


print("PDE Merton M2DBDP  Dim ", d, " layerSize " , layerSize,  " rescal " ,rescal, "T ", T , "batchsize ",batchSize, " batchSizeVal ", batchSizeVal, "num_epoch " , num_epoch, " num_epochExtNoLast ", num_epochExtNoLast  , "num_epochExtLast " , num_epochExtLast,"VOL " , sigScal, "initialLearningRateLast" , initialLearningRateLast , "initialLearningRateNoLast " , initialLearningRateNoLast)

# nest on ndt
for indt  in ndt:

    print("NBSTEP",indt)
    # create graph
    resol =  solv.PDEFNLSolve2OptGPU(model, T, indt[0], indt[1], theNetwork , theNetworkGam, initialLearningRateLast=initialLearningRateLast, initialLearningRateNoLast = initialLearningRateNoLast)

    baseFile = "MertonM2DBDPd"+str(d)+"nbNeur"+str(layerSize[0])+"nbHL"+str(len(layerSize))+"ndt"+str(indt[0])+str(indt[1])+"eta"+str(int(eta*100))
    plotFile = "pictures/"+baseFile
    saveFolder = "save/"

    Y0List = []
    for i in range(nTest):
        # train
        t0 = time.time()
        Y0, Z0, Gamma0  = resol.BuildAndtrainML( batchSize, batchSizeVal, num_epochExtNoLast=num_epochExtNoLast, num_epochExtLast= num_epochExtLast ,num_epoch=num_epoch,  nbOuterLearning=nbOuterLearning, thePlot= plotFile , baseFile = baseFile, saveDir= saveFolder)
        t1 = time.time()
        print(" NBSTEP", indt, " EstimMC Val is " , Y0,  " REAL IS ", model.Sol(0.,xInit),    " Z0 ", Z0," DERREAL IS  ",model.derSol(0.,xInit), "Gamma0 " , Gamma0,t1-t0)

        Y0List.append(Y0)
        print(Y0List)

    print("Y0", Y0List)
    yList = np.array(Y0List)
    yMean = np.mean(yList)
    print(" DNT ", indt , "MeanVal ", yMean, " Etyp ", np.sqrt(np.mean(np.power(yList-yMean,2.))))


args ['C:\\Users\\sojore\\anaconda3\\envs\\py37s\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\sojore\\AppData\\Roaming\\jupyter\\runtime\\kernel-f0a75d40-f5d1-4c6b-9047-39ffcb7b3402.json']
nbLayer  2
 WILL USE 2 THREADS 
REAL IS  [-0.506617]  DERIV [[0.2533085]]
PDE Merton M2DBDP  Dim  1  layerSize  [11 11]  rescal  1.0 T  1.0 batchsize  1000  batchSizeVal  10000 num_epoch  400  num_epochExtNoLast  10 num_epochExtLast  200 VOL  [1.] initialLearningRateLast 0.01 initialLearningRateNoLast  0.001
NBSTEP (120, 30)
nbStepGamStab 4
CREATED GAM  30
 Gam 0.0046353335  iout 0
 Gam 0.004592521  iout 2
 Gam 0.0045202156  iout 7
Projection, and derivative,  Learning decrease to  0.005
Projection, and derivative,  Learning decrease to  0.0025
 Gam 0.0043460457  iout 23
Projection, and derivative,  Learning decrease to  0.00125
 Gam 0.0043365564  iout 32
Projection, and derivative,  Learning decrease to  0.000625
Projection, and derivative,  Learning decrease to  0.0003125
Projecti

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\sojore\anaconda3\envs\py37s\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-775c1e9789d9>", line 74, in <module>
    Y0, Z0, Gamma0  = resol.BuildAndtrainML( batchSize, batchSizeVal, num_epochExtNoLast=num_epochExtNoLast, num_epochExtLast= num_epochExtLast ,num_epoch=num_epoch,  nbOuterLearning=nbOuterLearning, thePlot= plotFile , baseFile = baseFile, saveDir= saveFolder)
  File "C:\Users\sojore\Documents\pyhthon s\solvers\FullNonLinearBaseGPU.py", line 211, in BuildAndtrainML
    xPrev = self.model.getValues(iGam, self.TStepGam, self.xInit,batchSize)
  File "C:\Users\sojore\Documents\pyhthon s\models\Merton.py", line 94, in getValues
    np.sqrt(TStep*istep)*np.einsum('ij,j->ij',np.random.normal(size=[batchSize,self.d]),self.sigScal)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Tracebac

TypeError: object of type 'NoneType' has no len()

In [ ]:
#saving the model
model.save('MertonM2DBDPd_model.h5')

In [ ]:
#loading the saved model
from keras.models import load_model
new_MertonMDBDP_model=load_model('MertonM2DBDPd_model.h5')

In [ ]:
#getting the summary of the model
new_MertonMDBDP_model.summary()

In [ ]:
#implementing transfer_learn by freezing the inner layers of this new_MertonMDBDP_model ,so we can train it on the  
#MertonMDBDP_model_types_weights weights

In [ ]:
type(new_MertonMDBDP_model)

In [ ]:
#we always convert the functional models into sequential model and also remove the last layer (output layer)
MertonM2DBDPd_model_new=Sequential()
for layer in new_MertonMDBDP_model.layers[:-1]:#this code removes the last layer of the model
    MertonM2DBDPd_model_new.add(layer)

In [ ]:
#we can confirm the type of the model is Sequential or functional
type(MertonM2DBDPd_model_new)

In [ ]:
MertonM2DBDPd_model_new.summary()

In [ ]:
##freezing the layers so we dont train them again
for layer in MertonM2DBDPd_model_new.layers:
    layer.trainable=False #we set the hidden layers untrainable so we dont train them again

In [ ]:
#we now add a custom  last layer(Dense) to our transfered MertonM2DBDPd_model_new model with 2 output classes
MertonM2DBDPd_model_new.add(Dense(2,activation='softmax'))

In [ ]:
#validation of the added last layer
MertonM2DBDPd_model_new.summary()

In [ ]:
#we finnaly use this transefered MertonM2DBDPd_model_new model to load the already saved weights from  MertonMDBDP model
#thus we compare it perfomance

#we do the same for algorithms 1.MertonEMDBDP
#                              2.MertonPWG

#for this group Mertonxxx we implement the frozen models and load  the MertonMDBDP_model_types_weights

In [ ]:
#transfer  MertonM2DBDPd_model and train it on the MertonMDBDP_model_types_weights
MertonM2DBDPd_model_new.load_weights('MertonMDBDP_model_types_weights')

2.NoLeveragexxx

-in this groups of algorithms ,we gonna train one model e.g NoLeverageM2DBDP,we then save its weights
-train another model e.g NoLeverageEMDBDP ,freeze its hidden layers so it does not train again,
-transfer this model to load the saved weights from NoLeverageEMDBDP and compare its perfomance

In [48]:
#we now going to train another model NoLeverageM2DBDP ,save the model,load it again,freeze the hidden layers of the model,
#add an outer dense layer (output neurons=2) ,use this new model and load the NoLeverageEMDBDP_model_types_weights, to see its
#perfomance


In [30]:
import numpy as np
import tensorflow as tf
import os
import solvers  as solv
import networks as net
import models as mod
import multiprocessing
import sys
import time

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

_MULTIPROCESSING_CORE_COUNT = multiprocessing.cpu_count()
print("args", sys.argv)


nbLayer= 2  
print("nbLayer " ,nbLayer)
rescal=1.
T=1.

batchSize= 1000
batchSizeVal= 10000
num_epoch= 400 
num_epochExtNoLast =10
num_epochExtLast= 200
initialLearningRateLast = 1e-2
initialLearningRateNoLast = 1e-3
nbOuterLearning = 10
nTest = 10

lamb = np.array([1.5, 1.1, 2., 0.8, 0.5, 1.7, 0.9, 1., 0.9,1.5], dtype=np.float32)[0:9]
eta = 0.5
theta = np.array([0.1, 0.2, 0.3, 0.4, 0.25, 0.15, 0.18, 0.08, 0.91,0.4], dtype=np.float32)[0:9]
gamma = np.array([0.2, 0.15, 0.25, 0.31, 0.4, 0.35, 0.22, 0.4, 0.15,0.2], dtype=np.float32)[0:9]
kappa = np.array([1., 0.8, 1.1, 1.3, 0.95, 0.99, 1.02, 1.06, 1.6,1.], dtype=np.float32)[0:9]
sigma = np.ones(10, dtype=np.float32)[0:9]
d = lamb.shape[0] + 1

nbNeuron = d + 10
layerSize= nbNeuron*np.ones((nbLayer,), dtype=np.int32)

sigScal =  np.concatenate([np.array([1.], dtype=np.float32),gamma]).reshape((d))
muScal = np.concatenate([np.array([np.sum(theta*lamb)]),np.zeros((d-1), dtype=np.float32)]).reshape((d))
xyInit= np.concatenate([np.array([1.]),theta])   

# create the model
model = mod.ModelNoLeverage(xyInit, muScal, sigScal, T, theta, sigma, lamb, eta, gamma, kappa)

print(" WILL USE " + str(_MULTIPROCESSING_CORE_COUNT) + " THREADS ")
print("REAL IS ", model.Sol(0.,xyInit.reshape(1,d)), " DERIV", model.derSol(0.,xyInit.reshape(1,d)), " GAMMA", model.der2Sol(0.,xyInit.reshape(1,d)))

theNetwork = net.FeedForwardUZDZ(d,layerSize,tf.nn.tanh)

ndt = [120]


print("PDE No Leverage EMDBDP  Dim ", d, " layerSize " , layerSize,  " rescal " ,rescal, "T ", T , "batchsize ",batchSize, " batchSizeVal ", batchSizeVal, "num_epoch " , num_epoch, " num_epochExtNoLast ", num_epochExtNoLast  , "num_epochExtLast " , num_epochExtLast,"VOL " , sigScal, "initialLearningRateLast" , initialLearningRateLast , "initialLearningRateNoLast " , initialLearningRateNoLast)

# nest on ndt
for indt  in ndt:

    print("NBSTEP",indt)
    # create graph
    resol =  solv.PDEFNLSolveSimpleLSExp(model, T, indt, theNetwork , initialLearningRateLast=initialLearningRateLast, initialLearningRateNoLast = initialLearningRateNoLast)
        
    baseFile = "NoLeverageEMDBDPd"+str(d)+"nbNeur"+str(layerSize[0])+"nbHL"+str(len(layerSize))+"ndt"+str(indt)+"eta"+str(int(eta*100))
    plotFile = "pictures/"+baseFile
    saveFolder = "save/"

    Y0List = []
    for i in range(nTest):
        # train
        t0 = time.time()
        Y0, Z0, Gamma0  = resol.BuildAndtrainML( batchSize, batchSizeVal, num_epochExtNoLast=num_epochExtNoLast, num_epochExtLast= num_epochExtLast ,num_epoch=num_epoch,  nbOuterLearning=nbOuterLearning, thePlot= plotFile ,  baseFile = baseFile, saveDir= saveFolder)
        t1 = time.time()
        print(" NBSTEP", indt, " EstimMC Val is " , Y0,  " REAL IS ", model.Sol(0.,xyInit.reshape((1,d))),    " Z0 ", Z0," DERREAL IS  ",model.derSol(0.,xyInit.reshape((1,d))), "Gamma0 " , Gamma0,  " GAMMA", model.der2Sol(0.,xyInit.reshape(1,d)),t1-t0)

        Y0List.append(Y0)
        print(Y0List)

    print("Y0", Y0List)
    yList = np.array(Y0List)
    yMean = np.mean(yList)
    print(" DNT ", indt , "MeanVal ", yMean, " Etyp ", np.sqrt(np.mean(np.power(yList-yMean,2.))))


args ['C:\\Users\\sojore\\anaconda3\\envs\\py37s\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\sojore\\AppData\\Roaming\\jupyter\\runtime\\kernel-15a0e8bb-a378-44a0-8223-90f3ae60377d.json']
nbLayer  2
 WILL USE 2 THREADS 
REAL IS  [-0.27509173]  DERIV [[0.13754587 0.03860316 0.04560767 0.19347028 0.0391144  0.01102995
  0.07188345 0.02499711 0.01326371 0.10095984]]  GAMMA [[[-0.06877293 -0.01930158 -0.02280383 -0.09673514 -0.0195572
   -0.00551498 -0.03594172 -0.01249855 -0.00663185 -0.05047992]
  [-0.01930158  0.25916169 -0.00640005 -0.02714936 -0.00548886
   -0.00154781 -0.01008728 -0.0035078  -0.00186127 -0.01416752]
  [-0.02280383 -0.00640005  0.15778577 -0.03207558 -0.00648481
   -0.00182866 -0.01191761 -0.00414429 -0.002199   -0.01673821]
  [-0.09673514 -0.02714936 -0.03207558  0.29628993 -0.02750891
   -0.0077573  -0.05055518 -0.01758031 -0.00932828 -0.07100442]
  [-0.0195572  -0.00548886 -0.00648481 -0.02750891  0.05675259
   -0.00156831 -0.01022087 -0.00355426

TypeError: Tensor objects are only iterable when eager execution is enabled. To iterate over this tensor use tf.map_fn.

In [ ]:
#lets save the weights,and the above trained model 
#saving the weights/parameters
model.save_weights('NoLeverageEMDBDP_model_types_weights.h5')#this will save the weights of the NoLeverageEMDBDP model trained above
#which we want
#to load them using a different algorithm such as NoLeverageEMDBDP and see how it perfoms of these weights

In [7]:
# we are first executing the code below so as to get a NoLeverageM2DBDP_model,which we can then save it,freeze the hidden layers
#then transfer the new model to load the NoLeverageEMDBDP_model_types_weights
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import solvers  as solv
import networks as net
import models as mod
import multiprocessing
import sys
import time

_MULTIPROCESSING_CORE_COUNT = multiprocessing.cpu_count()
print("args", sys.argv)

nbLayer= 2  
print("nbLayer " ,nbLayer)
sig = 1 
print("Sig ",  sig)
rescal=1.
muScal =0.
T=1.

batchSize= 1000
batchSizeVal= 10000
num_epoch=400 
num_epochExtNoLast =10
num_epochExtLast= 200
initialLearningRateLast = 1e-2
initialLearningRateNoLast = 1e-3
nbOuterLearning =10
nTest = 10


lamb = np.array([1.5, 1.1, 2., 0.8, 0.5, 1.7, 0.9, 1., 0.9, 1.5], dtype=np.float32)[0:9]
eta = 0.5
theta = np.array([0.1, 0.2, 0.3, 0.4, 0.25, 0.15, 0.18, 0.08, 0.91, 0.4], dtype=np.float32)[0:9]
gamma = np.array([0.2, 0.15, 0.25, 0.31, 0.4, 0.35, 0.22, 0.4, 0.15, 0.2], dtype=np.float32)[0:9]
kappa = np.array([1., 0.8, 1.1, 1.3, 0.95, 0.99, 1.02, 1.06, 1.6, 0.1], dtype=np.float32)[0:9]
sigma = np.ones(9, dtype=np.float32)[0:9]
d = lamb.shape[0] + 1
nbNeuron = d + 10
sigScal =  np.concatenate([np.array([1.], dtype=np.float32),gamma]).reshape((d))

muScal = np.concatenate([np.array([np.sum(theta*lamb)]),np.zeros((d-1), dtype=np.float32)]).reshape((d))
layerSize= nbNeuron*np.ones((nbLayer,), dtype=np.int32)

xyInit= np.concatenate([np.array([1]),theta])   

# create the model
model = mod.ModelNoLeverage(xyInit, muScal, sigScal, T, theta, sigma, lamb, eta, gamma, kappa)

print(" WILL USE " + str(_MULTIPROCESSING_CORE_COUNT) + " THREADS ")
print("REAL IS ", model.Sol(0.,xyInit.reshape(1,d)), " DERIV", model.derSol(0.,xyInit.reshape(1,d)), " GAMMA", model.der2Sol(0.,xyInit.reshape(1,d)))

theNetwork = net.FeedForwardGam(d,layerSize,tf.nn.tanh)
theNetworkGam = net.FeedForwardGam(d,layerSize,tf.nn.tanh)

ndt = [ (120,30) ] 

print("PDE No Leverage M2DBDP  Dim ", d, " layerSize " , layerSize,  " rescal " ,rescal, "T ", T , "batchsize ",batchSize, " batchSizeVal ", batchSizeVal, "num_epoch " , num_epoch, " num_epochExtNoLast ", num_epochExtNoLast  , "num_epochExtLast " , num_epochExtLast,  "VOL " , sigScal, "initialLearningRateLast" , initialLearningRateLast , "initialLearningRateNoLast " , initialLearningRateNoLast)

# nest on ndt
for indt  in ndt:

    print("NBSTEP",indt)
    # create graph
    resol =  solv.PDEFNLSolve2OptGPU(model, T, indt[0], indt[1], theNetwork , theNetworkGam, initialLearningRateLast=initialLearningRateLast, initialLearningRateNoLast = initialLearningRateNoLast)
        
    baseFile = "NoLeverageM2DBDPd"+str(d)+"nbNeur"+str(layerSize[0])+"nbHL"+str(len(layerSize))+"ndt"+str(indt[0])+str(indt[1])+"eta"+str(int(eta*100))
    plotFile = "pictures/"+baseFile
    saveFolder = "save/"

    Y0List = []
    for i in range(nTest):
        # train
        t0 = time.time()
        Y0, Z0, Gamma0  = resol.BuildAndtrainML( batchSize, batchSizeVal, num_epochExtNoLast=num_epochExtNoLast, num_epochExtLast= num_epochExtLast ,num_epoch=num_epoch,  nbOuterLearning=nbOuterLearning, thePlot= plotFile , baseFile = baseFile, saveDir= saveFolder)
        t1 = time.time()
        print(" NBSTEP", indt, " EstimMC Val is " , Y0,  " REAL IS ", model.Sol(0.,xyInit.reshape(1,d)),    " Z0 ", Z0," DERREAL IS  ",model.derSol(0.,xyInit.reshape(1,d)), "Gamma0 " , Gamma0, " GAMMA", model.der2Sol(0.,xyInit.reshape(1,d)),t1-t0)

        Y0List.append(Y0)
        print(Y0List)

    print("Y0", Y0List)
    yList = np.array(Y0List)
    yMean = np.mean(yList)
    print(" DNT ", indt , "MeanVal ", yMean, " Etyp ", np.sqrt(np.mean(np.power(yList-yMean,2.))))


args ['C:\\Users\\sojore\\anaconda3\\envs\\py37s\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\sojore\\AppData\\Roaming\\jupyter\\runtime\\kernel-f0a75d40-f5d1-4c6b-9047-39ffcb7b3402.json']
nbLayer  2
Sig  1
 WILL USE 2 THREADS 
REAL IS  [-0.27509173]  DERIV [[0.13754587 0.03860316 0.04560767 0.19347028 0.0391144  0.01102995
  0.07188345 0.02499711 0.01326371 0.10095984]]  GAMMA [[[-0.06877293 -0.01930158 -0.02280383 -0.09673514 -0.0195572
   -0.00551498 -0.03594172 -0.01249855 -0.00663185 -0.05047992]
  [-0.01930158  0.25916169 -0.00640005 -0.02714936 -0.00548886
   -0.00154781 -0.01008728 -0.0035078  -0.00186127 -0.01416752]
  [-0.02280383 -0.00640005  0.15778577 -0.03207558 -0.00648481
   -0.00182866 -0.01191761 -0.00414429 -0.002199   -0.01673821]
  [-0.09673514 -0.02714936 -0.03207558  0.29628993 -0.02750891
   -0.0077573  -0.05055518 -0.01758031 -0.00932828 -0.07100442]
  [-0.0195572  -0.00548886 -0.00648481 -0.02750891  0.05675259
   -0.00156831 -0.01022087 -0.0

CREATED GAM  30
 Gam 3.2573266e-06  iout 0
 Gam 1.3486886e-06  iout 1
 Gam 8.6670593e-07  iout 2
 Gam 7.5633966e-07  iout 3
 Gam 4.4232112e-07  iout 4
 Gam 2.1290529e-07  iout 5
 Gam 1.3017322e-07  iout 7
 Gam 7.785034e-08  iout 8
 Gam 7.417928e-08  iout 13
 Gam 4.374364e-08  iout 17
 Gam 3.8678728e-08  iout 19
 Gam 1.9020163e-08  iout 22
 Gam 1.762136e-08  iout 38
Projection, and derivative,  Learning decrease to  0.005
 Gam 8.998467e-09  iout 41
 Gam 5.829438e-09  iout 43
 Gam 5.6072413e-09  iout 47
 Gam 5.5204037e-09  iout 52
 Gam 3.1692016e-09  iout 53
Projection, and derivative,  Learning decrease to  0.0025
 Gam 2.3604756e-09  iout 72
 Gam 2.1545215e-09  iout 75
 Gam 1.0185558e-09  iout 78
Projection, and derivative,  Learning decrease to  0.00125
 Gam 3.2552738e-10  iout 91
 Gam 1.8279885e-10  iout 93
Projection, and derivative,  Learning decrease to  0.000625
 Gam 1.2088419e-10  iout 132
 Gam 8.66347e-11  iout 133
Projection, and derivative,  Learning decrease to  0.0003125
 Ga

INFO:tensorflow:Restoring parameters from save/NoLeverageM2DBDPd10nbNeur20nbHL2ndt12030eta50Gam_30


TypeError: Tensor objects are only iterable when eager execution is enabled. To iterate over this tensor use tf.map_fn.

In [11]:
pwd

'C:\\Users\\sojore\\Documents\\pyhthon s'

In [ ]:
#saving the model
model.save('NoLeverageM2DBDP_model.h5')

In [ ]:
#loading the saved model
from keras.models import load_model
new_NoLeverageM2DBDP_model=load_model('NoLeverageM2DBDP_model.h5')

In [ ]:
#we always convert the functional models into sequential model and also remove the last layer (output layer)
NoLeverageM2DBDP_model_new=Sequential()
for layer in new_NoLeverageM2DBDP_model.layers[:-1]:#this code removes the last layer of the model
    NoLeverageM2DBDP_model_new.add(layer)

In [ ]:
##freezing the layers so we dont train them again
for layer in NoLeverageM2DBDP_model_new.layers:
    layer.trainable=False #we set the hidden layers untrainable so we dont train them again

In [ ]:
#we now add a custom  last layer(Dense) to our transfered NoLeverageM2DBD_model_new model with 2 output classes
NoLeverageM2DBDP_model_new.add(Dense(2,activation='softmax'))

In [ ]:
#we finnaly use this transefered NoLeverageM2DBDP_model_new model to load the already saved weights from  NoLeverageEMDBDP model
#thus we compare it perfomance

#we do the same for algorithms 1.NoLeverageMDBDP
#                              2.NoLeveragePWG

#for this group NoLeverageMxxx we implement the frozen models and load  the NoLeverageEMDBDP_model_types_weights

In [ ]:
#transfer  NoLeverageM2DBDP_model and train it on the NoLeverageEMDBDP_model_types_weights
NoLeverageM2DBDP_model_new.load_weights('NoLeverageEMDBDP_model_types_weights')

3.OneAssetxxx

-in this groups of algorithms ,we gonna train one model e.g OneAssetEMDBDP,we then save its weights -train another model e.g OneAssetM2DBDP ,freeze its hidden layers so it does not train again, -transfer this model to load the saved weights from OneAssetEMDBDP and compare its perfomance

In [59]:
import numpy as np
import tensorflow as tf
import os
import solvers  as solv
import networks as net
import models as mod
import multiprocessing
import sys
import time

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

_MULTIPROCESSING_CORE_COUNT = multiprocessing.cpu_count()
print("args", sys.argv)

nbLayer= 2  
print("nbLayer " ,nbLayer)
rescal=1.
T=1.

batchSize= 1000
batchSizeVal= 10000
num_epoch= 400 
num_epochExtNoLast =10
num_epochExtLast= 200
initialLearningRateLast = 1e-2
initialLearningRateNoLast = 1e-3
nbOuterLearning = 10
nTest = 10

lamb = np.array([1.], dtype=np.float32)
eta = 0.5
theta = np.array([0.4], dtype=np.float32) 
gamma = np.array([0.4], dtype=np.float32) 
kappa = np.array([1.], dtype=np.float32) 
sigma = np.array([1.], dtype=np.float32)
rho = np.array([-0.7], dtype=np.float32)
d = lamb.shape[0] + 1
nbNeuron = d + 10
sigScal =  np.concatenate([np.array([1.], dtype=np.float32),gamma]).reshape((d))
muScal = np.concatenate([np.array([np.sum(theta*lamb)]),np.zeros((d-1), dtype=np.float32)]).reshape((d))

layerSize= nbNeuron*np.ones((nbLayer,), dtype=np.int32)
xyInit= np.concatenate([np.array([1]),theta])  

# create the model
model = mod.ModelOneAsset(xyInit, muScal, sigScal, T, theta, sigma, lamb, eta, gamma, kappa, rho)

print(" WILL USE " + str(_MULTIPROCESSING_CORE_COUNT) + " THREADS ")
print("REAL IS ", model.Sol(0.,xyInit.reshape(1,d)), " DERIV", model.derSol(0.,xyInit.reshape(1,d)), " GAMMA", model.der2Sol(0.,xyInit.reshape(1,d)))

theNetwork = net.FeedForwardUZDZ(d,layerSize,tf.nn.tanh)

ndt = [120]


print("PDE OneAsset EMDBDP  Dim ", d, " layerSize " , layerSize,  " rescal " ,rescal, "T ", T , "batchsize ",batchSize, " batchSizeVal ", batchSizeVal, "num_epoch " , num_epoch, " num_epochExtNoLast ", num_epochExtNoLast  , "num_epochExtLast " , num_epochExtLast,"VOL " , sigScal, "initialLearningRateLast" , initialLearningRateLast , "initialLearningRateNoLast " , initialLearningRateNoLast)

# nest on ndt
for indt  in ndt:

    print("NBSTEP",indt)
    # create graph
    resol =  solv.PDEFNLSolveSimpleLSExp(model, T, indt, theNetwork , initialLearningRateLast=initialLearningRateLast, initialLearningRateNoLast = initialLearningRateNoLast)
        
    baseFile = "OneAssetEMDBDPd"+str(d)+"nbNeur"+str(layerSize[0])+"nbHL"+str(len(layerSize))+"ndt"+str(indt)+"eta"+str(int(eta*100))
    plotFile = "pictures/"+baseFile
    saveFolder = "save/"

    Y0List = []
    for i in range(nTest):
        # train
        t0 = time.time()
        Y0, Z0, Gamma0  = resol.BuildAndtrainML( batchSize, batchSizeVal, num_epochExtNoLast=num_epochExtNoLast, num_epochExtLast= num_epochExtLast ,num_epoch=num_epoch,  nbOuterLearning=nbOuterLearning, thePlot= plotFile ,  baseFile = baseFile, saveDir= saveFolder)
        t1 = time.time()
        print(" NBSTEP", indt, " EstimMC Val is " , Y0,  " REAL IS ", model.Sol(0.,xyInit.reshape((1,d))),    " Z0 ", Z0," DERREAL IS  ",model.derSol(0.,xyInit.reshape((1,d))), "Gamma0 " , Gamma0," GAMMA", model.der2Sol(0.,xyInit.reshape(1,d)),t1-t0)

        Y0List.append(Y0)
        print(Y0List)

    print("Y0", Y0List)
    yList = np.array(Y0List)
    yMean = np.mean(yList)
    print(" DNT ", indt , "MeanVal ", yMean, " Etyp ", np.sqrt(np.mean(np.power(yList-yMean,2.))))


args ['C:\\Users\\sojore\\anaconda3\\envs\\py37s\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\sojore\\AppData\\Roaming\\jupyter\\runtime\\kernel-15a0e8bb-a378-44a0-8223-90f3ae60377d.json']
nbLayer  2
 WILL USE 2 THREADS 
REAL IS  [-0.5361144]  DERIV [[0.2680572  0.16555405]]  GAMMA [[[-0.1340286  -0.08277703]
  [-0.08277703  0.22920763]]]
PDE OneAsset EMDBDP  Dim  2  layerSize  [12 12]  rescal  1.0 T  1.0 batchsize  1000  batchSizeVal  10000 num_epoch  400  num_epochExtNoLast  10 num_epochExtLast  200 VOL  [1.  0.4] initialLearningRateLast 0.01 initialLearningRateNoLast  0.001
NBSTEP 120


KeyboardInterrupt: 

In [ ]:
#lets save the weights,and the above trained model 
#saving the weights/parameters
model.save_weights('OneAssetEMDBDP_model_types_weights.h5')#this will save the weights of the OneAssetEMDBDP model trained above
#which we want
#to load them using a different algorithm such as OneAssetM2DBDP and see how it perfoms of these weights

In [ ]:
# we are first executing the code below so as to get a OneAssetM2DBDP_model,which we can then save it,freeze the hidden layers
#then transfer the new model to load the OneAssetEMDBDP_model_types_weights

In [56]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import solvers  as solv
import networks as net
import models as mod
import multiprocessing
import sys
import time

 

_MULTIPROCESSING_CORE_COUNT = multiprocessing.cpu_count()
print("args", sys.argv)

nbLayer= 2  
print("nbLayer " ,nbLayer)
sig = 1 
print("Sig ",  sig)
rescal=1.
T=1.

batchSize= 1000
batchSizeVal= 10000
num_epoch=400 
num_epochExtNoLast =10
num_epochExtLast= 200
initialLearningRateLast = 1e-2
initialLearningRateNoLast = 1e-3
nbOuterLearning =10
nTest = 10

lamb = np.array([1.], dtype=np.float32)
eta = 0.5
theta = np.array([0.4], dtype=np.float32) 
gamma = np.array([0.4], dtype=np.float32) 
kappa = np.array([1.], dtype=np.float32) 
sigma = np.array([1.], dtype=np.float32)
rho = np.array([-0.7], dtype=np.float32)
d = lamb.shape[0] + 1
nbNeuron = d + 10
sigScal =  np.concatenate([np.array([1.], dtype=np.float32),gamma]).reshape((d))

muScal = np.concatenate([np.array([np.sum(theta*lamb)]),np.zeros((d-1), dtype=np.float32)]).reshape((d))

layerSize= nbNeuron*np.ones((nbLayer,), dtype=np.int32)
xyInit= np.concatenate([np.array([1]),theta])  

# create the model
model = mod.ModelOneAsset(xyInit, muScal, sigScal, T, theta, sigma, lamb, eta, gamma, kappa, rho)

print(" WILL USE " + str(_MULTIPROCESSING_CORE_COUNT) + " THREADS ")
print("REAL IS ", model.Sol(0.,xyInit.reshape(1,d)), " DERIV", model.derSol(0.,xyInit.reshape(1,d)), " GAMMA", model.der2Sol(0.,xyInit.reshape(1,d)))

theNetwork = net.FeedForwardUZDZ(d,layerSize,tf.nn.tanh)
theNetworkGam = net.FeedForwardGam(d,layerSize,tf.nn.tanh)

ndt = [(120,30) ] 
print("PDE OneAsset M2DBDP  Dim ", d, " layerSize " , layerSize,  " rescal " ,rescal, "T ", T , "batchsize ",batchSize, " batchSizeVal ", batchSizeVal, "num_epoch " , num_epoch, " num_epochExtNoLast ", num_epochExtNoLast  , "num_epochExtLast " , num_epochExtLast,  "VOL " , sigScal, "initialLearningRateLast" , initialLearningRateLast , "initialLearningRateNoLast " , initialLearningRateNoLast)

# nest on ndt
for indt  in ndt:

    print("NBSTEP",indt)
    # create graph
    resol =  solv.PDEFNLSolve2OptGPU(model, T, indt[0], indt[1], theNetwork , theNetworkGam, initialLearningRateLast=initialLearningRateLast, initialLearningRateNoLast = initialLearningRateNoLast)
        
    baseFile = "OneAssetM2DBDPd"+str(d)+"nbNeur"+str(layerSize[0])+"nbHL"+str(len(layerSize))+"ndt"+str(indt[0])+str(indt[1])+"eta"+str(int(eta*100))
    plotFile = "pictures/"+baseFile
    saveFolder = "save/"

    Y0List = []
    for i in range(nTest):
        # train
        t0 = time.time()
        Y0, Z0, Gamma0  = resol.BuildAndtrainML( batchSize, batchSizeVal, num_epochExtNoLast=num_epochExtNoLast, num_epochExtLast= num_epochExtLast ,num_epoch=num_epoch,  nbOuterLearning=nbOuterLearning, thePlot= plotFile , baseFile = baseFile, saveDir= saveFolder)
        t1 = time.time()
        print(" NBSTEP", indt, " EstimMC Val is " , Y0,  " REAL IS ", model.Sol(0.,xyInit.reshape(1,d)),    " Z0 ", Z0," DERREAL IS  ",model.derSol(0.,xyInit.reshape(1,d)), "Gamma0 " , Gamma0, " GAMMA", model.der2Sol(0.,xyInit.reshape(1,d)),t1-t0)

        Y0List.append(Y0)
        print(Y0List)

    print("Y0", Y0List)
    yList = np.array(Y0List)
    yMean = np.mean(yList)
    print(" DNT ", indt , "MeanVal ", yMean, " Etyp ", np.sqrt(np.mean(np.power(yList-yMean,2.))))


args ['C:\\Users\\sojore\\anaconda3\\envs\\py37s\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\sojore\\AppData\\Roaming\\jupyter\\runtime\\kernel-15a0e8bb-a378-44a0-8223-90f3ae60377d.json']
nbLayer  2
Sig  1
 WILL USE 2 THREADS 
REAL IS  [-0.5361144]  DERIV [[0.2680572  0.16555405]]  GAMMA [[[-0.1340286  -0.08277703]
  [-0.08277703  0.22920763]]]
PDE OneAsset M2DBDP  Dim  2  layerSize  [12 12]  rescal  1.0 T  1.0 batchsize  1000  batchSizeVal  10000 num_epoch  400  num_epochExtNoLast  10 num_epochExtLast  200 VOL  [1.  0.4] initialLearningRateLast 0.01 initialLearningRateNoLast  0.001
NBSTEP (120, 30)
nbStepGamStab 4
CREATED GAM  30
 Gam 0.00014217863  iout 0


KeyboardInterrupt: 

In [ ]:
#saving the model
model.save('OneAssetM2DBDP_model.h5')

In [ ]:
#loading the saved model
from keras.models import load_model
new_OneAssetM2DBDP_model=load_model('OneAssetM2DBDP_model.h5')

In [ ]:
#we always convert the functional models into sequential model and also remove the last layer (output layer)
OneAssetM2DBDP_model_new=Sequential()
for layer in new_OneAssetM2DBDP_model.layers[:-1]:#this code removes the last layer of the model
    OneAssetM2DBDP_model_new.add(layer)

In [ ]:
##freezing the layers so we dont train them again
for layer in OneAssetM2DBDP_model_new.layers:
    layer.trainable=False #we set the hidden layers untrainable so we dont train them again

In [ ]:
#we now add a custom  last layer(Dense) to our transfered OneAssetM2DBDP_model_new model with 2 output classes
OneAssetM2DBDP_model_new.add(Dense(2,activation='softmax'))

In [ ]:
#we finnaly use this transefered OneAssetM2DBDP_model_new model to load the already saved weights from  OneAssetEMDBDP model
#thus we compare it perfomance

#we do the same for algorithms 1.OneAssetMDBDP
#                              2.OneAssetPWG

In [ ]:
#transfer  OneAssetM2DBDP_model and train it on the OneAssetEMDBDP_model_types_weights
OneAssetM2DBDP_model_new.load_weights('OneAssetEMDBDP_model_types_weights')

In [65]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import solvers  as solv
import networks as net
import models as mod
import multiprocessing
import sys
import time

_MULTIPROCESSING_CORE_COUNT = multiprocessing.cpu_count()
print("args", sys.argv)

nbLayer= 2  
print("nbLayer " ,nbLayer)

rescal=1.
T=1.

batchSize= 1000
batchSizeVal= 50000 
num_epoch= 50 
num_epochExtNoLast = 200
initialLearningRateLast = 1e-2
initialLearningRateNoLast = 1e-3
nbOuterLearning =10
nTest = 10

d = 1
nbNeuron = 11
sigScal = np.ones(d, dtype=np.float32)
muScal = np.zeros(d, dtype=np.float32)
nu = 1

layerSize= nbNeuron*np.ones((nbLayer,), dtype=np.int32)

xInit = np.ones(d, dtype=np.float32) 

# create the model
model = mod.Burgers(xInit, muScal, sigScal, T, nu)

print(" WILL USE " + str(_MULTIPROCESSING_CORE_COUNT) + " THREADS ")

print("REAL IS ", model.Sol(0.,xInit.reshape(1,d),1000000), " DERIV", model.derSol(0.,xInit.reshape(1,d),1000000))

theNetwork = net.FeedForwardUZ(d,layerSize,tf.nn.tanh)

ndt = [ 120 ] 


print("PDE Burgers HJE  Dim ", d, " layerSize " , layerSize,  " rescal " ,rescal, "T ", T , "batchsize ",batchSize, " batchSizeVal ", batchSizeVal, "num_epoch " , num_epoch, " num_epochExtNoLast ", num_epochExtNoLast  ,"VOL " , sigScal, "initialLearningRateNoLast " , initialLearningRateNoLast)

# nest on ndt
for indt  in ndt:

    print("NBSTEP",indt)
    # create graph
    resol =  solv.SemiHJE(model, T, indt, theNetwork, initialLearningRateNoLast = initialLearningRateNoLast)
    
    baseFile = "BurgersHJEd"+str(d)+"nbNeur"+str(layerSize[0])+"nbHL"+str(len(layerSize))+"ndt"+str(indt)
    plotFile = "pictures/"+baseFile
    saveFolder = "save/"

    Y0List = []
    for i in range(nTest):
        # train
        t0 = time.time()
        Y0, Z0  = resol.BuildAndtrainML( batchSize, batchSizeVal, num_epochExtNoLast=num_epochExtNoLast ,num_epoch=num_epoch,  nbOuterLearning=nbOuterLearning, thePlot= plotFile , baseFile = baseFile, saveDir= saveFolder)
        t1 = time.time()
        print(" NBSTEP", indt, " EstimMC Val is " , Y0,  " REAL IS ", model.Sol(0.,xInit.reshape(1,d)),    " Z0 ", Z0," DERREAL IS  ",model.derSol(0.,xInit.reshape(1,d)),t1-t0)

        Y0List.append(Y0)
        print(Y0List)

    print("Y0", Y0List)
    yList = np.array(Y0List)
    yMean = np.mean(yList)
    print(" DNT ", indt , "MeanVal ", yMean, " Etyp ", np.sqrt(np.mean(np.power(yList-yMean,2.))))


args ['C:\\Users\\sojore\\anaconda3\\envs\\py37s\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\sojore\\AppData\\Roaming\\jupyter\\runtime\\kernel-15a0e8bb-a378-44a0-8223-90f3ae60377d.json']
nbLayer  2
 WILL USE 2 THREADS 
REAL IS  [0.43470901]  DERIV [[-0.54530285]]
PDE Burgers HJE  Dim  1  layerSize  [11 11]  rescal  1.0 T  1.0 batchsize  1000  batchSizeVal  50000 num_epoch  50  num_epochExtNoLast  200 VOL  [1.] initialLearningRateNoLast  0.001
NBSTEP 120


Exception ignored in: <function TF_Buffer.<lambda> at 0x000002099C814168>
Traceback (most recent call last):
  File "C:\Users\sojore\anaconda3\envs\py37s\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 1269, in <lambda>
    __del__ = lambda self: None
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
#lets save the weights,and the above trained model 
#saving the weights/parameters
model.save_weights('BurgersHJE_model_types_weights.h5')#this will save the weights of the BurgersHJE model trained above
#which we want
#to load them using a different algorithm such as BurgersHPW and see how it perfoms of these weights

In [69]:
# we are then executing the code below so as to get a BurgersHPW_model,which we can then save it,freeze the hidden layers
#then transfer the new model to load the BurgersHJE_model_types_weights
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import solvers  as solv
import networks as net
import models as mod
import multiprocessing
import sys
import time


_MULTIPROCESSING_CORE_COUNT = multiprocessing.cpu_count()
print("args", sys.argv)

nbLayer= 2
print("nbLayer " ,nbLayer)

T=1.

batchSize= 1000 
batchSizeVal= 50000 
num_epoch= 50 
num_epochExtNoLast = 100
num_epochExtLast= 1000 
initialLearningRateLast = 1e-2 
initialLearningRateNoLast = 1e-3
nbOuterLearning =10
nTest = 10

d = 1
nbNeuron = 11
sigScal = np.ones(d)
muScal = np.zeros(d)
nu = 1

layerSize= nbNeuron*np.ones((nbLayer,), dtype=np.int32)

xInit = np.ones(d) #np.ones(d) 

# create the model
model = mod.Burgers(xInit, muScal, sigScal, T, nu)

print(" WILL USE " + str(_MULTIPROCESSING_CORE_COUNT) + " THREADS ")
print("REAL IS ", model.Sol(0.,xInit.reshape(1,d),1000000), " DERIV", model.derSol(0.,xInit.reshape(1,d),1000000))

theNetworkUZ = net.FeedForwardUDU(d,layerSize,tf.nn.tanh)

ndt = [120] 

print("PDE Burgers HPW Dim ", d, " layerSize " , layerSize,   "T ", T , "batchsize ",batchSize, " batchSizeVal ", batchSizeVal, "num_epoch " , num_epoch, " num_epochExtNoLast ", num_epochExtNoLast  , "num_epochExtLast " , num_epochExtLast,  "VOL " , sigScal, "initialLearningRateLast" , initialLearningRateLast , "initialLearningRateNoLast " , initialLearningRateNoLast)

# nest on ndt
for indt  in ndt:

    print("NBSTEP",indt)
    # create graph
    resol =  solv.HPW(xInit, model, T, indt, theNetworkUZ, initialLearningRate= initialLearningRateLast,initialLearningRateStep =  initialLearningRateNoLast)
        
    baseFile = "BurgersHPW"+str(d)+"nbNeur"+str(layerSize[0])+"nbHL"+str(len(layerSize))+"ndt"+str(indt)
    plotFile = "C:\\Users\\sojore\\Documents\\pyhthon s/"+baseFile #make sure to change the directory 
    saveFolder = "save/"

    Y0List = []
    for i in range(nTest):
        # train
        t0 = time.time()
        Y0, Z0  = resol.BuildAndtrainML( batchSize, batchSizeVal, num_epoch=num_epoch, num_epochExt=num_epochExtNoLast,  nbOuterLearning=nbOuterLearning, thePlot= plotFile , baseFile = baseFile, saveDir= saveFolder)
        t1 = time.time()
        print(" NBSTEP", indt, " EstimMC Val is " , Y0,  " REAL IS ", model.Sol(0.,xInit.reshape(1,d),1000000),    " Z0 ", Z0,t1-t0)

        Y0List.append(Y0)
        print(Y0List)

    print("Y0", Y0List)
    yList = np.array(Y0List)
    yMean = np.mean(yList)
    print(" DNT ", indt , "MeanVal ", yMean, " Etyp ", np.sqrt(np.mean(np.power(yList-yMean,2.))))


args ['C:\\Users\\sojore\\anaconda3\\envs\\py37s\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\sojore\\AppData\\Roaming\\jupyter\\runtime\\kernel-15a0e8bb-a378-44a0-8223-90f3ae60377d.json']
nbLayer  2
 WILL USE 2 THREADS 
REAL IS  [0.43098218]  DERIV [[-0.54596591]]
PDE Burgers HPW Dim  1  layerSize  [11 11] T  1.0 batchsize  1000  batchSizeVal  50000 num_epoch  50  num_epochExtNoLast  100 num_epochExtLast  1000 VOL  [1.] initialLearningRateLast 0.01 initialLearningRateNoLast  0.001
NBSTEP 120
Graph for projection took  2.0258026123046875
Loss is  0.055037953
Loss is  0.026047349
Loss is  0.01027732
Loss is  0.0048738606
Loss is  0.0025993004
Loss is  0.0018919173
Loss is  0.0012241035
Loss is  0.0009446024
Loss is  0.00081393117
Loss is  0.00067497353
Loss is  0.00052242284
Loss is  0.00046589575
Loss is  0.00057286647
Loss is  0.0004235851
Loss is  0.00041435505
Loss is  0.00038085942
Loss is  0.00038096216
Loss is  0.00034791353
Loss is  0.00030084915
Loss is  0.000

KeyboardInterrupt: 

In [15]:
pwd

'C:\\Users\\sojore\\Documents\\pyhthon s'

In [ ]:
#saving the model
model.save('BurgersHPW_model.h5')

In [ ]:
#loading the saved model
from keras.models import load_model
new_BurgersHPW_model=load_model('BurgersHPW_model.h5')

In [ ]:
#we always convert the functional models into sequential model and also remove the last layer (output layer)
BurgersHPW_model_new=Sequential()
for layer in new_BurgersHPW_model.layers[:-1]:#this code removes the last layer of the model
    BurgersHPW_model_new.add(layer)

In [ ]:
##freezing the layers so we dont train them again
for layer in BurgersHPW_model_new.layers:
    layer.trainable=False #we set the hidden layers untrainable so we dont train them again

In [ ]:
#we now add a custom  last layer(Dense) to our transfered BurgersHPW_model_new model with 2 output classes
BurgersHPW_model_new.add(Dense(2,activation='softmax'))

In [ ]:
#transfer  BurgersHPW_model and train it on the BurgersHJE_model_types_weights
BurgersHPW_model_new.load_weights('BurgersHJE_model_types_weights')

5.BoundedFNLxxx

-in this group of algorithms we gonna train BoundedFNLEMDBDP model,save its weights
-we then train BoundedFNLPWG model,freeze its hidden layers,then transfer it to load the saved weights
-also compare its perfomance

In [ ]:
#we now going to train another model BoundedFNLPWG ,save the model,load it again,freeze the hidden layers of the model,
#add an outer dense layer (output neurons=2) ,use this new model and load the BoundedFNLEMDBDP_model_types_weights, to see its
#perfomance


In [ ]:
#lets save the weights,and the above trained model 
#saving the weights/parameters
model.save_weights('BoundedFNLEMDBDP_model_types_weights.h5')#this will save the weights of the BoundedFNLEMDBDP model trained above
#which we want
#to load them using a different algorithm such as BoundedFNLEMDBDP and see how it perfoms of these weights

In [ ]:
#saving the model
model.save('BoundedFNLPWG_model.h5')

In [ ]:
#loading the saved model
from keras.models import load_model
new_BoundedFNLPWG_model=load_model('BoundedFNLPWG_model.h5')

In [ ]:
#getting the summary of the model
new_BoundedFNLPWG_model.summary()

In [ ]:
#we always convert the functional models into sequential model and also remove the last layer (output layer)
BoundedFNLPWG_model_new=Sequential()
for layer in new_BoundedFNLPWG_model.layers[:-1]:#this code removes the last layer of the model
    BoundedFNLPWG_model_new.add(layer)

In [ ]:
#we can confirm the type of the model is Sequential or functional
type(BoundedFNLPWG_model_new)

In [ ]:
##freezing the layers so we dont train them again
for layer in BoundedFNLPWG_model_new.layers:
    layer.trainable=False #we set the hidden layers untrainable so we dont train them again

In [ ]:
#we now add a custom  last layer(Dense) to our transfered BoundedFNLPWG_model_new model with 2 output classes
BoundedFNLPWG_model_new.add(Dense(2,activation='softmax'))

In [ ]:
#we finnaly use this transefered BoundedFNLPWG_model_new model to load the already saved weights from  BoundedFNLEMDBDP model
#thus we compare it perfomance

#we do the same for algorithms 1.BoundedFNLM2DBDP
#                              2.BoundedFNLMDBDP

#for this group BoundedFNLxxx we implement the frozen models and load  the BoundedFNLEMDBDP_model_types_weights

In [ ]:
#transfer  BoundedFNLPWG_model and train it on the BoundedFNLEMDBDP_model_types_weights
BoundedFNLPWG_model_new.load_weights('BoundedFNLEMDBDP_model_types_weights')

NOTE THAT THE ABOVE PROCEDURES CAN BE USED TO TRANSFER ANY MODEL TO LOAD DIFFERENT WEIGHTS

In [82]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import solvers  as solv
import networks as net
import models as mod
import multiprocessing
import sys
import time


_MULTIPROCESSING_CORE_COUNT = multiprocessing.cpu_count()
print("args", sys.argv)

nbLayer= 3 
print("nbLayer " ,nbLayer)
T=1.

batchSize= 1000
batchSizeVal= 10000
num_epoch=400 
num_epochExtNoLast =10
num_epochExtLast= 200
initialLearningRateLast = 1e-2
initialLearningRateNoLast = 1e-3
nbOuterLearning =10
nTest = 10

d = 15
nbNeuron = d + 10
sigScal =  0.5*np.ones(d)
muScal = np.zeros(d)
lamb = 0.5

layerSize= nbNeuron*np.ones((nbLayer,), dtype=np.int32)

xyInit = np.ones(d) 

# create the model
model = mod.ModelMongeAmpere(xyInit, muScal, sigScal, T, lamb, d)

print(" WILL USE " + str(_MULTIPROCESSING_CORE_COUNT) + " THREADS ")
print("REAL IS ", model.Sol(0.,xyInit.reshape(1,d)), " DERIV", model.derSol(0.,xyInit.reshape(1,d)), " GAMMA", model.der2Sol(0.,xyInit.reshape(1,d)))

theNetwork = net.FeedForwardUZDZ(d,layerSize,tf.nn.tanh)

ndt = [120] 

print("PDE Monge Ampere EMDBDP  Dim ", d, " layerSize " , layerSize,   "T ", T , "batchsize ",batchSize, " batchSizeVal ", batchSizeVal, "num_epoch " , num_epoch, " num_epochExtNoLast ", num_epochExtNoLast  , "num_epochExtLast " , num_epochExtLast,  "VOL " , sigScal, "initialLearningRateLast" , initialLearningRateLast , "initialLearningRateNoLast " , initialLearningRateNoLast)

# nest on ndt
for indt  in ndt:

    print("NBSTEP",indt)
    # create graph
    resol =  solv.PDEFNLSolveSimpleLSExp(model, T, indt, theNetwork , initialLearningRateLast=initialLearningRateLast, initialLearningRateNoLast = initialLearningRateNoLast)
        
    baseFile = "MongeAmpereEMDBDPd"+str(d)+"nbNeur"+str(layerSize[0])+"nbHL"+str(len(layerSize))+"ndt"+str(indt)
    plotFile = "pictures/"+baseFile
    saveFolder = "save/"

    Y0List = []
    for i in range(nTest):
        # train
        t0 = time.time()
        Y0, Z0, Gamma0  = resol.BuildAndtrainML( batchSize, batchSizeVal, num_epochExtNoLast=num_epochExtNoLast, num_epochExtLast= num_epochExtLast ,num_epoch=num_epoch,  nbOuterLearning=nbOuterLearning, thePlot= plotFile , baseFile = baseFile, saveDir= saveFolder)
        t1 = time.time()
        print(" NBSTEP", indt, " EstimMC Val is " , Y0,  " REAL IS ", model.Sol(0.,xyInit.reshape(1,d)),    " Z0 ", Z0," DERREAL IS  ",model.derSol(0.,xyInit.reshape(1,d)), "Gamma0 " , Gamma0, " GAMMA", model.der2Sol(0.,xyInit.reshape(1,d)),t1-t0)

        Y0List.append(Y0)
        print(Y0List)

    print("Y0", Y0List)
    yList = np.array(Y0List)
    yMean = np.mean(yList)
    print(" DNT ", indt , "MeanVal ", yMean, " Etyp ", np.sqrt(np.mean(np.power(yList-yMean,2.))))


args ['C:\\Users\\sojore\\anaconda3\\envs\\py37s\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\sojore\\AppData\\Roaming\\jupyter\\runtime\\kernel-15a0e8bb-a378-44a0-8223-90f3ae60377d.json']
nbLayer  3
 WILL USE 2 THREADS 
REAL IS  [0.25575373]  DERIV [[0.17245241 0.17245241 0.17245241 0.17245241 0.17245241 0.17245241
  0.17245241 0.17245241 0.17245241 0.17245241 0.17245241 0.17245241
  0.17245241 0.17245241 0.17245241]]  GAMMA [[[0.22206882 0.04961642 0.04961642 0.04961642 0.04961642 0.04961642
   0.04961642 0.04961642 0.04961642 0.04961642 0.04961642 0.04961642
   0.04961642 0.04961642 0.04961642]
  [0.04961642 0.22206882 0.04961642 0.04961642 0.04961642 0.04961642
   0.04961642 0.04961642 0.04961642 0.04961642 0.04961642 0.04961642
   0.04961642 0.04961642 0.04961642]
  [0.04961642 0.04961642 0.22206882 0.04961642 0.04961642 0.04961642
   0.04961642 0.04961642 0.04961642 0.04961642 0.04961642 0.04961642
   0.04961642 0.04961642 0.04961642]
  [0.04961642 0.04961642 0.

KeyboardInterrupt: 

In [96]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import solvers  as solv
import networks as net
import models as mod
import multiprocessing
import sys
import time

_MULTIPROCESSING_CORE_COUNT = multiprocessing.cpu_count()
print("args", sys.argv)

nbLayer= 2
print("nbLayer " ,nbLayer)

T=1.

batchSize= 1000 
batchSizeVal= 50000 
num_epoch= 50 
num_epochExtNoLast = 100
num_epochExtLast= 1000 
initialLearningRateLast = 1e-2 
initialLearningRateNoLast = 1e-3
nbOuterLearning = 50
nTest = 1 
d = 1
nbNeuron = 11
sigScal = np.ones(d)
muScal = np.zeros(d)
nu = 1

layerSize= nbNeuron*np.ones((nbLayer,), dtype=np.int32)

xyInit = np.ones(d) 

# create the model
model = mod.Burgers(xyInit, muScal, sigScal, T, nu)

print(" WILL USE " + str(_MULTIPROCESSING_CORE_COUNT) + " THREADS ")
print("REAL IS ", model.Sol(0.,xyInit.reshape(1,d),1000000), " DERIV", model.derSol(0.,xyInit.reshape(1,d),1000000))

theNetwork = net.FeedForwardUZ(d,layerSize,tf.nn.tanh)

ndt = [120] 

print("PDE Burgers MDBDP Dim ", d, " layerSize " , layerSize,   "T ", T , "batchsize ",batchSize, " batchSizeVal ", batchSizeVal, "num_epoch " , num_epoch, " num_epochExtNoLast ", num_epochExtNoLast  , "num_epochExtLast " , num_epochExtLast,  "VOL " , sigScal, "initialLearningRateLast" , initialLearningRateLast , "initialLearningRateNoLast " , initialLearningRateNoLast)

# nest on ndt
for indt  in ndt:

    print("NBSTEP",indt)
    # create graph
    resol =  solv.PDESLSolveBaseGPU(model, T, indt, theNetwork,initialLearningRateLast=initialLearningRateLast, initialLearningRateNoLast = initialLearningRateNoLast)
        
    baseFile = "BurgersMDBDP"+str(d)+"nbNeur"+str(layerSize[0])+"nbHL"+str(len(layerSize))+"ndt"+str(indt)
    plotFile = "pictures/"+baseFile
    saveFolder = "save/"

    Y0List = []
    for i in range(nTest):
        # train
        t0 = time.time()
        Y0, Z0  = resol.BuildAndtrainML( batchSize, batchSizeVal, num_epochExtNoLast=num_epochExtNoLast, num_epochExtLast= num_epochExtLast ,num_epoch=num_epoch,  nbOuterLearning=nbOuterLearning, thePlot= plotFile , baseFile = baseFile, saveDir= saveFolder)
        t1 = time.time()
        print(" NBSTEP", indt, " EstimMC Val is " , Y0,  " REAL IS ", model.Sol(0.,xyInit.reshape(1,d)),    " Z0 ", Z0,t1-t0)

        Y0List.append(Y0)
        print(Y0List)

    print("Y0", Y0List)
    yList = np.array(Y0List)
    yMean = np.mean(yList)
    print(" DNT ", indt , "MeanVal ", yMean, " Etyp ", np.sqrt(np.mean(np.power(yList-yMean,2.))))


args ['C:\\Users\\sojore\\anaconda3\\envs\\py37s\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\sojore\\AppData\\Roaming\\jupyter\\runtime\\kernel-15a0e8bb-a378-44a0-8223-90f3ae60377d.json']
nbLayer  2
 WILL USE 2 THREADS 
REAL IS  [0.43310275]
PDE Burgers Splitting Dim  1  layerSize  [11 11] T  1.0 batchsize  1000  batchSizeVal  50000 num_epoch  50  num_epochExtNoLast  100 num_epochExtLast  1000 VOL  [1.] initialLearningRateLast 0.01 initialLearningRateNoLast  0.001
NBSTEP 120


ValueError: Shapes must be equal rank, but are 2 and 3
	From merging shape 1 with other shapes. for 'sub_2/x' (op: 'Pack') with input shapes: [?], [?,1], [?,1,1].

In [106]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import solvers  as solv
import networks as net
import models as mod
import multiprocessing
import sys
import time

# 2 optimization version
# Non linearity in   "uD2u"
# suppress verbose
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  

_MULTIPROCESSING_CORE_COUNT = multiprocessing.cpu_count()
print("args", sys.argv)

nbLayer= 3 
print("nbLayer " ,nbLayer)
T=1.

batchSize= 1000
batchSizeVal= 10000
num_epoch=400 
num_epochExtNoLast =10
num_epochExtLast= 200
initialLearningRateLast = 1e-2
initialLearningRateNoLast = 1e-3
nbOuterLearning =10
nTest = 10

d = 15
nbNeuron = d + 10
sigScal =  0.5*np.ones(d)
muScal = np.zeros(d)
lamb = 0.5

layerSize= nbNeuron*np.ones((nbLayer,), dtype=np.int32)

xyInit = np.ones(d) 

# create the model
model = mod.ModelMongeAmpere(xyInit, muScal, sigScal, T, lamb, d)

print(" WILL USE " + str(_MULTIPROCESSING_CORE_COUNT) + " THREADS ")
print("REAL IS ", model.Sol(0.,xyInit.reshape(1,d)), " DERIV", model.derSol(0.,xyInit.reshape(1,d)), " GAMMA", model.der2Sol(0.,xyInit.reshape(1,d)))

theNetwork = net.FeedForwardUDU(d,layerSize,tf.nn.tanh)
theNetworkGam = net.FeedForwardGam(d,layerSize,tf.nn.tanh)

ndt = [(120,30)  ] 

print("PDE Monge Ampere MDBDP  Dim ", d, " layerSize " , layerSize,   "T ", T , "batchsize ",batchSize, " batchSizeVal ", batchSizeVal, "num_epoch " , num_epoch, " num_epochExtNoLast ", num_epochExtNoLast  , "num_epochExtLast " , num_epochExtLast,  "VOL " , sigScal, "initialLearningRateLast" , initialLearningRateLast , "initialLearningRateNoLast " , initialLearningRateNoLast)

# nest on ndt
for indt  in ndt:

    print("NBSTEP",indt)
    # create graph
    resol =  solv.PDEFNLSolve2OptZGPU(model, T, indt[0], indt[1], theNetwork , theNetworkGam, initialLearningRateLast=initialLearningRateLast, initialLearningRateNoLast = initialLearningRateNoLast)
        
    baseFile = "MongeAmpereMDBDPd"+str(d)+"nbNeur"+str(layerSize[0])+"nbHL"+str(len(layerSize))+"ndt"+str(indt[0])+str(indt[1])
    plotFile = "pictures/"+baseFile
    saveFolder = "save/"

    Y0List = []
    for i in range(nTest):
        # train
        t0 = time.time()
        Y0, Z0, Gamma0  = resol.BuildAndtrainML( batchSize, batchSizeVal, num_epochExtNoLast=num_epochExtNoLast, num_epochExtLast= num_epochExtLast ,num_epoch=num_epoch,  nbOuterLearning=nbOuterLearning, thePlot= plotFile , baseFile = baseFile, saveDir= saveFolder)
        t1 = time.time()
        print(" NBSTEP", indt, " EstimMC Val is " , Y0,  " REAL IS ", model.Sol(0.,xyInit.reshape(1,d)),    " Z0 ", Z0," DERREAL IS  ",model.derSol(0.,xyInit.reshape(1,d)), "Gamma0 " , Gamma0, " GAMMA", model.der2Sol(0.,xyInit.reshape(1,d)),t1-t0)

        Y0List.append(Y0)
        print(Y0List)

    print("Y0", Y0List)
    yList = np.array(Y0List)
    yMean = np.mean(yList)
    print(" DNT ", indt , "MeanVal ", yMean, " Etyp ", np.sqrt(np.mean(np.power(yList-yMean,2.))))


args ['C:\\Users\\sojore\\anaconda3\\envs\\py37s\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\sojore\\AppData\\Roaming\\jupyter\\runtime\\kernel-15a0e8bb-a378-44a0-8223-90f3ae60377d.json']
nbLayer  3
 WILL USE 2 THREADS 
REAL IS  [0.25575373]  DERIV [[0.17245241 0.17245241 0.17245241 0.17245241 0.17245241 0.17245241
  0.17245241 0.17245241 0.17245241 0.17245241 0.17245241 0.17245241
  0.17245241 0.17245241 0.17245241]]  GAMMA [[[0.22206882 0.04961642 0.04961642 0.04961642 0.04961642 0.04961642
   0.04961642 0.04961642 0.04961642 0.04961642 0.04961642 0.04961642
   0.04961642 0.04961642 0.04961642]
  [0.04961642 0.22206882 0.04961642 0.04961642 0.04961642 0.04961642
   0.04961642 0.04961642 0.04961642 0.04961642 0.04961642 0.04961642
   0.04961642 0.04961642 0.04961642]
  [0.04961642 0.04961642 0.22206882 0.04961642 0.04961642 0.04961642
   0.04961642 0.04961642 0.04961642 0.04961642 0.04961642 0.04961642
   0.04961642 0.04961642 0.04961642]
  [0.04961642 0.04961642 0.

KeyboardInterrupt: 